###-----------------------

In [780]:
import numpy as np

In [781]:
blok = 1     # 
numpad = 10   # 9 hücreden oluşuyor, hücrelerin adları numpadle aynı
boy = 30       #robotun boyu ve eni 30cm olarak düşünülebilir
engelli = 0
#durum  -1 kullanılmıyor
#        0: gidilmedi
#        1: gidildi
#        2: engelli (gidilemez)
durum = np.zeros((10),dtype='int')
#her hücrenin kendi adıyla girilebilmesi için [0] kullanım dışı (-1)
durum[0] = -1

#hücrelerin puanları
#engelliyse 0 
#her hücrenin başlangıç puanı 10
puan = 10*np.ones((10),dtype='int')
#her hücrenin kendi adıyla girilebilmesi için [0] kullanım dışı (-1)
puan[0] = -1

#her gidişte artan
ziyaret = np.zeros((10),dtype='int')

In [782]:
#current cell: hücrem
hücrem = 0

In [783]:
import turtle
from turtle import *
vn = turtle.Screen()
robo = turtle.Turtle()
engel = turtle.Turtle()
koor = (3*boy) + 1
#sıfır,sıfır sol üst köşede
mode('world')
setworldcoordinates(0,koor,koor,0)

reset()

In [784]:
#directions 
#0 - east 
#90 - south
#180 - west
#270 - north

def dön(yön):
    turtle.setheading(yön)
    robo.setheading(yön)
    return

In [785]:
#Alanı çiz
robo.reset()
def çiz(x,y):
    
    robo.penup()
    robo.setpos(x,y)
    robo.pendown()
    robo.forward(boy)

    return

çiz(0,0)
çiz(0,10)
çiz(0,20)
çiz(0,30)

dön(270)

çiz(30,30)
çiz(20,30)
çiz(10,30)
çiz(0,30)


In [786]:
# değişken haline getirilecek
#cell numpad kurallarıyla
def cell(number):
    return {
        1: (5,25),
        2: (15,25),
        3: (25,25),
        4: (5,15),
        5: (15,15),
        6: (25,15),
        7: (5,5),
        8: (15,5),
        9: (25,5)
    } [number]


#beta
def sıfırdurumlukomşuseç(komşulistesi):
    for yer in komşulistesi:
        if durum[yer]==0:
            return yer
        


#durduğu hücreye göre yönünü gideceği hücreye çeviriyor    
def yönbul(hücre):
    return {
        1: [ 0,270],
        2: [180,0,270],
        3: [180,270],
        4: [90,0,270],
        5: [90,180,0,270],
        6: [90,180,270],
        7: [90,0],
        8: [90,180,0],
        9: [90,180]
    } [hücre]


def komşubul(current):
    return {
        1: [2,4],
        2: [1,3,5],
        3: [2,6],
        4: [1,5,7],
        5: [2,4,6,8],
        6: [3,5,9],
        7: [4,8],
        8: [5,7,9],
        9: [6,8]
    } [current]
    
    return

def doğruyönedön(sayı):
    ind = komşubul(hücrem)
    for index, item in enumerate(ind):
        if item==sayı:
    #inde = ind.index(sayı)
            dön(yönbul(hücrem)[index])
    return

In [788]:
#beta
#durumları sıfır olanlara ziyaret
def uzaklaraGit():
    for sıfır in durum:
        if sıfır==0:
            return sıfır
    bitti = True    
    return

In [789]:
def engelkoy(sayı):
    global durum,puan,engelli
    if sayı==1:
        return
    engelli +=1
    engel.setpos(cell(sayı))
    durum[sayı]= 2
    puan[sayı] = -1000  
    engel.stamp()

In [790]:
#engel oluşturuyor
engel.penup()

engel.color("yellow")
engel.shape("circle")
engel.shapesize(3,3)



In [791]:
#engel konuyor
engelkoy(2)
engelkoy(1) # 1 numaraya istese de engel koyamaz

In [792]:
turtle.reset()
turtle.penup()

turtle.color("pink")
turtle.shape("turtle")

turtle.shapesize(3,2)
turtle.setpos(5,5)



In [793]:
def Git(sayı):
    global hücrem,durum,puan,ziyaret

    doğruyönedön(sayı)
    turtle.setpos(cell(sayı))
    
    #Güncelleme
    hücrem = sayı
    ziyaret[sayı] += 1
    durum[sayı]=1
    puan[sayı]-=0.5
        

In [794]:
def puanaGöreKomşuSeç():
    global puanlistesi
    global komşulistesi
    puanlistesi = []
    komşulistesi = komşubul(hücrem)
    for yer in komşulistesi:
        puanlistesi.append(puan[yer])
    d = dict(zip(komşulistesi,puanlistesi))
    return sorted(d, key=d.__getitem__, reverse=True)[0]

In [795]:
def tara2():
    bitti = False
    while not bitti:
        Git(puanaGöreKomşuSeç())
        if sum(durum) == (2*engelli) -1 + (9-engelli):
            bitti = True
            return

In [800]:
#başlangıç
#1 numaralı cell engelli olamaz
dön(270)
hücrem = 1
turtle.setpos(cell(1))
durum[1]=1
puan[1]-=0.5
ziyaret[1]+=1
turtle.speed(1)


In [801]:
#main
turtle.pendown()
tara2()


# Alanı genişlet